In [ ]:
!nvidia-smi

Wed Oct 12 04:48:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Set Up Environment

1) Google Drive에 Mount를 진행함.

2) 필요 package install

3) 공식 StyleGAN2-ADA git clone

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.6 kB/s 
     |████████████████████████████████| 17.4 MB 63.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-pytho

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (57/57), done.


# Find Your Files

Mount된 이미지 데이터 세트의 개수를 확인하고 압축을 해제함.

```
/content/drive/MyDrive/data
```


In [ ]:
!ls /content/drive/MyDrive/data/gan/images

anime_5k  anime_face_15k  celeba_hq_15k  Optic1000_512


In [ ]:
!unzip -qq "/content/drive/MyDrive/data/gan/images/celeba_hq_15k.zip" -d "/content/drive/MyDrive/data/gan/images/celeba_hq_15k"

In [ ]:
img_dir_name = "celeba_hq_15k"

In [ ]:
import glob
import os

DIR = "/content/drive/MyDrive/data/gan/images/" + img_dir_name
print (len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

15000


# Convert Your Images

dataset_tool.py를 활용하여 활용하고자 하는 데이터 세트의 이미지 사이즈를 resize하고 Drive에 저장함.

In [ ]:
!rm -R /content/drive/MyDrive/data/gan/dataset/circuit/*

rm: cannot remove '/content/drive/MyDrive/data/gan/dataset/circuit/*': No such file or directory


In [ ]:
W, H = 256, 256

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/data/gan/images/celeba_hq_15k/ --dest /content/drive/MyDrive/data/gan/dataset/circuit --width {W} --height {H}

100% 15000/15000 [10:30<00:00, 23.78it/s]


# Training from scratch

처음부터 훈련시키거나 특정 위치의 pkl을 활용하여 훈련시키기 위한 코드

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset/circuit"
SNAP = 8

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

In [ ]:
!/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap 25 --resume /content/drive/MyDrive/data/gan/experiments/00007-circuit-auto1/network-snapshot-000500.pkl --outdir /content/drive/MyDrive/data/gan/experiments --data /content/drive/MyDrive/data/gan/dataset/circuit

# Transfer Learning

FFHQ pkl 파일을 기반으로 훈련시키고자 하는 데이터 세트를 학습시킬 수 있음.

이를 통해 빠르고 좋은 이미지 생성이 가능

In [ ]:
#!wget -P /content/drive/MyDrive/data/gan/pretrain https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/ffhq-res256-mirror-paper256-noaug.pkl

In [ ]:
# RESUME = "/content/drive/MyDrive/data/gan/pretrain/ffhq-res256-mirror-paper256-noaug.pkl"
# or
RESUME = 'ffhq256'

FREEZED = 4
BATCH = 32

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
#NETWORK = "network-snapshot-000200.pkl"
#RESUME = os.path.join(EXPERIMENTS, "00000-circuit-auto1", NETWORK)

# If you want to shrink the set_up snapshot image grid, then change the number in training_loop.py in training dir
# gw = np.clip(1024 // training_set.image_shape[2], 2, 32)
# gh = np.clip(1024 // training_set.image_shape[1], 2, 32)

DATA = "/content/drive/MyDrive/data/gan/dataset/circuit"
SNAP = 8

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --batch {BATCH} --data {DATA} --freezed {FREEZED}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 8,
  "network_snapshot_ticks": 8,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/data/gan/dataset/circuit",
    "use_labels": false,
    "max_size": 15000,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {
      "freeze_layers": 4
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
   

# Style Mixing

Source Image(row), Reference Image(col)에 따라 Style Mixing을 진행함.

이때, style에 대한 부분을 일부 고정시킴으로써 다양한 조합을 만들어낼 수 있음.

이를 통해 reference image에 걸맞는 스타일의 이미지 변환이 가능하게 함.


In [ ]:
import os
create_dir_path = "/content/drive/MyDrive/data/gan/experiments/style_mixing_2"
os.makedirs(create_dir_path, exist_ok=True)

In [ ]:
output_dir = create_dir_path
rows = "1,2,3,4"
cols = "7,8,9,10"
styles = "0-6"
pkl_file_path = "/content/drive/MyDrive/data/gan/experiments/00003-circuit-auto1-batch32-resumeffhq256-freezed4/network-snapshot-000384.pkl"

cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/style_mixing.py --outdir {output_dir} --rows {rows} --cols {cols} --styles {styles} --network {pkl_file_path}"
!{cmd}

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00003-circuit-auto1-batch32-resumeffhq256-freezed4/network-snapshot-000384.pkl"...
Generating W vectors...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Generating images...
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating style-mixed images...
Saving images...
Saving image grid...


style_mixing.py 파일 내 마지막을 아래와 같이 수정함으로써 다수의 결과 이미지가 중복되지 않도록 한다.

canvas.save(f'{outdir}/grid_row_{row_seeds[0]}_to_{row_seeds[-1]}_col_{col_seeds[0]}_to_{col_seeds[-1]}_style_{col_styles[0]}_to_{col_styles[-1]}.png')

In [ ]:
import numpy as np
r = ','.join(map(str, list(np.arange(21,31))))
c = ','.join(map(str, list(np.arange(31,41))))
print(r, c)

21,22,23,24,25,26,27,28,29,30 31,32,33,34,35,36,37,38,39,40


In [ ]:
output_dir = create_dir_path

#rows = "1,2,3,4,6,7,8,9,10"
#cols = "11,12,13,14,15,16,17,18,19,20"
rows = r
cols = c
pkl_file_path = "/content/drive/MyDrive/data/gan/experiments/00003-circuit-auto1-batch32-resumeffhq256-freezed4/network-snapshot-000384.pkl"
styles = ["0", "0-1", "0-2", "0-3", "0-4", "0-5", "0-6"]

for style in styles:
  cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/style_mixing.py --outdir {output_dir} --rows {rows} --cols {cols} --styles {style} --network {pkl_file_path}"
  !{cmd}

Loading networks from "/content/drive/MyDrive/data/gan/experiments/00003-circuit-auto1-batch32-resumeffhq256-freezed4/network-snapshot-000384.pkl"...
Generating W vectors...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Generating images...
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating style-mixed images...
Saving images...
Saving image grid...
Loading networks from "/content/drive/MyDrive/data/gan/experiments/00003-circuit-auto1-batch32-resumeffhq256-freezed4/network-snapshot-000384.pkl"...
Generating W vectors...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Generating images...
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating style-mixed images...
Saving images...
Saving image grid...
Loading networks from "/content/drive/MyDrive/data/gan/experiments/00003-circuit-auto1-batch32-resumeffhq256-freezed4/network-snapshot-000384.pkl"...
Generating W vectors...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Generating images...
Settin